In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from PIL import Image, ImageFilter
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T
import torch.nn.functional as F

USE_GPU = True
dtype = torch.float32 # We will be using float throughout this tutorial.

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss.
# print_every = 100
print('using device:', device)

using device: cpu


In [6]:
def flatten(x):
    N = x.shape[0]
    return x.view(N, -1)

class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)


In [7]:
gray_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # Normalize the data
    transforms.Grayscale(1)
])

gray_trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=gray_transform)
gray_trainloader = torch.utils.data.DataLoader(gray_trainset, batch_size=64,
                                          shuffle=True)

gray_testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=gray_transform)
gray_testloader = torch.utils.data.DataLoader(gray_testset, batch_size=64,
                                         shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
def gray_check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on train set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

test_accuracies = []
train_accuracies = []

def gray_train(model, optimizer, epochs=10):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(gray_trainloader):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

        print(f'Epoch {e}, loss = {loss.item()}')
        test_accuracies.append(gray_check_accuracy(gray_testloader, model))
        train_accuracies.append(gray_check_accuracy(gray_trainloader, model))
        print()

In [11]:
learning_rate = 0.001

fully_connected_model = nn.Sequential(
    Flatten(),
    nn.Linear(1024, 1024),
    nn.ReLU(),
    nn.Linear(1024, 1024),
    nn.ReLU(),
    nn.Linear(1024, 10)
)

optimizer = optim.SGD(fully_connected_model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)
gray_train(fully_connected_model, optimizer, epochs = 50)

Epoch 0, loss = 2.1699349880218506
Checking accuracy on test set
Got 2265 / 10000 correct (22.65)
Checking accuracy on train set
Got 11152 / 50000 correct (22.30)

Epoch 1, loss = 2.120011329650879
Checking accuracy on test set
Got 2673 / 10000 correct (26.73)
Checking accuracy on train set
Got 13268 / 50000 correct (26.54)

Epoch 2, loss = 2.016875982284546
Checking accuracy on test set
Got 3042 / 10000 correct (30.42)
Checking accuracy on train set
Got 15127 / 50000 correct (30.25)

Epoch 3, loss = 2.010275363922119
Checking accuracy on test set
Got 3340 / 10000 correct (33.40)
Checking accuracy on train set
Got 16641 / 50000 correct (33.28)

Epoch 4, loss = 1.6363526582717896
Checking accuracy on test set
Got 3568 / 10000 correct (35.68)
Checking accuracy on train set
Got 17841 / 50000 correct (35.68)

Epoch 5, loss = 1.8993582725524902
Checking accuracy on test set
Got 3696 / 10000 correct (36.96)
Checking accuracy on train set
Got 18856 / 50000 correct (37.71)

Epoch 6, loss = 1.7

In [128]:
from scipy import stats, fft

class CIFARDataset(torch.utils.data.Dataset):
    def __init__(self, train, dataset_path='/cifar10', freq_radius = 128):
        self.dataset = torchvision.datasets.CIFAR10(root='./data', train=train,
                                        download=True)
        self.freq_radius = freq_radius
        self.train = train

    def __getitem__(self, index):
        (img, label) = self.dataset[index]
        transformed_img = np.mean(np.array(img), axis = -1)           # convert into grayscale
        transformed_img = np.reshape(transformed_img, (1024, ))
        transformed_img = fft.dct(transformed_img, type = 2)
        transformed_img = np.concatenate((transformed_img[-self.freq_radius:], transformed_img[:self.freq_radius]))
        transformed_img = stats.zscore(transformed_img)
        return torch.from_numpy(transformed_img), label

    def __len__(self):
        return len(self.dataset)

dct_trainset = CIFARDataset(train = True)
dct_trainloader = torch.utils.data.DataLoader(dct_trainset, batch_size=64,
                                          shuffle=True)

dct_testset = CIFARDataset(train = False)
dct_testloader = torch.utils.data.DataLoader(dct_testset, batch_size=64,
                                         shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [129]:
def dct_check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on train set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

test_accuracies = []
train_accuracies = []

def dct_train(model, optimizer, epochs=10):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(dct_trainloader):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

        print(f'Epoch {e}, loss = {loss.item()}')
        test_accuracies.append(dct_check_accuracy(dct_testloader, model))
        train_accuracies.append(dct_check_accuracy(dct_trainloader, model))
        print()

In [135]:
learning_rate = 0.001

dct_model = nn.Sequential(
    Flatten(),
    nn.Linear(256, 256),
    nn.ReLU(),
    nn.Linear(256, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

optimizer = optim.SGD(dct_model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)
dct_train(dct_model, optimizer, epochs = 50)

Epoch 0, loss = 2.216207504272461
Checking accuracy on test set
Got 2044 / 10000 correct (20.44)
Checking accuracy on train set
Got 10249 / 50000 correct (20.50)

Epoch 1, loss = 2.362769365310669
Checking accuracy on test set
Got 2136 / 10000 correct (21.36)
Checking accuracy on train set
Got 10928 / 50000 correct (21.86)

Epoch 2, loss = 1.9851371049880981
Checking accuracy on test set
Got 2430 / 10000 correct (24.30)
Checking accuracy on train set
Got 12310 / 50000 correct (24.62)

Epoch 3, loss = 2.097755193710327
Checking accuracy on test set
Got 2574 / 10000 correct (25.74)
Checking accuracy on train set
Got 12984 / 50000 correct (25.97)

Epoch 4, loss = 1.872762680053711
Checking accuracy on test set
Got 2637 / 10000 correct (26.37)
Checking accuracy on train set
Got 13397 / 50000 correct (26.79)

Epoch 5, loss = 1.942753553390503
Checking accuracy on test set
Got 2720 / 10000 correct (27.20)
Checking accuracy on train set
Got 13923 / 50000 correct (27.85)

Epoch 6, loss = 1.944

In [ ]:
learning_rate = 3e-4

optimizer = optim.SGD(dct_model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)
dct_train(dct_model, optimizer, epochs = 150)

Epoch 0, loss = 1.3866915702819824
Checking accuracy on test set
Got 3978 / 10000 correct (39.78)
Checking accuracy on train set
Got 21087 / 50000 correct (42.17)

Epoch 1, loss = 1.8054126501083374
Checking accuracy on test set
Got 4001 / 10000 correct (40.01)
Checking accuracy on train set
Got 21073 / 50000 correct (42.15)

Epoch 2, loss = 1.5109609365463257
Checking accuracy on test set
Got 3962 / 10000 correct (39.62)
Checking accuracy on train set
Got 21117 / 50000 correct (42.23)

Epoch 3, loss = 1.5482374429702759
Checking accuracy on test set
Got 4022 / 10000 correct (40.22)
Checking accuracy on train set
Got 21246 / 50000 correct (42.49)

Epoch 4, loss = 1.5881743431091309
Checking accuracy on test set
Got 3972 / 10000 correct (39.72)
Checking accuracy on train set
Got 21001 / 50000 correct (42.00)

Epoch 5, loss = 1.5952140092849731
Checking accuracy on test set
Got 3982 / 10000 correct (39.82)
Checking accuracy on train set
Got 21165 / 50000 correct (42.33)

Epoch 6, loss = 